In [2]:
import useful
data, nyu2_train, nyu2_test  = useful.load_zip('nyu_data.zip')

Loading dataset zip file...Loaded (50688).


In [19]:
from data import DepthDataset, RandomHorizontalFlip, ToTensorTest, ToTensorTrain
transformed_training = DepthDataset(data, nyu2_train, transform=ToTensorTrain())
transformed_testing = DepthDataset(data, nyu2_test, transform=None)

In [58]:
import numpy as np
from useful import normalize_depth
normalize_depth(transformed_training[7]['depth'])

tensor([[[3.3117, 3.3117, 3.3117,  ..., 6.0714, 6.0714, 6.0714],
         [3.3117, 3.3117, 3.3117,  ..., 6.0714, 6.0714, 6.0714],
         [3.3117, 3.3117, 3.3117,  ..., 6.0714, 6.0714, 6.0714],
         ...,
         [3.8060, 3.8060, 3.8060,  ..., 9.4444, 9.4444, 9.4444],
         [3.8060, 3.8060, 3.8060,  ..., 9.4444, 9.4444, 9.4444],
         [3.8060, 3.8060, 3.8060,  ..., 9.4444, 9.4444, 9.4444]]])

In [56]:
normalize_depth(torch.clamp(np.array(transformed_testing[5]['depth'], np.float32)/10,10,1000))

array([[ 3.631082 ,  3.631082 ,  3.6324012, ...,  9.727626 ,  9.727626 ,
         9.727626 ],
       [ 3.629764 ,  3.631082 ,  3.6324012, ...,  9.727626 ,  9.727626 ,
         9.727626 ],
       [ 3.628447 ,  3.629764 ,  3.631082 , ...,  9.727626 ,  9.727626 ,
         9.737099 ],
       ...,
       [ 2.7570994,  2.75786  ,  2.7586207, ..., 10.111223 , 10.111223 ,
        10.111223 ],
       [ 2.75786  ,  2.7586207,  2.7586207, ..., 10.111223 , 10.111223 ,
        10.111223 ],
       [ 2.7586207,  2.7586207,  2.759382 , ..., 10.111223 , 10.111223 ,
        10.111223 ]], dtype=float32)

In [14]:
transformed_testing[0]['depth']

tensor([[[275.2000, 275.2000, 275.2000,  ..., 224.2000, 224.2000, 224.2000],
         [275.1000, 275.2000, 275.2000,  ..., 224.2000, 224.2000, 224.2000],
         [275.1000, 275.1000, 275.2000,  ..., 224.2000, 224.2000, 224.2000],
         ...,
         [219.8000, 219.8000, 219.7000,  ..., 208.1000, 208.1000, 208.0000],
         [219.7000, 219.7000, 219.7000,  ..., 208.1000, 208.1000, 208.1000],
         [219.7000, 219.7000, 219.7000,  ..., 208.1000, 208.1000, 208.1000]]])

In [ ]:
import numpy as np
from zipfile import ZipFile
from io import BytesIO
from PIL import Image
from useful import normalize_depth
# Load test data
def load_test_data():    
    print("Loading test data...")
    def extract_zip(input_zip):
        input_zip=ZipFile(input_zip)
        return {name: input_zip.read(name) for name in input_zip.namelist()}
    data = extract_zip('nyu_test.zip')
    rgb = np.load(BytesIO(data['eigen_test_rgb.npy']))
    depth = np.load(BytesIO(data['eigen_test_depth.npy']))
    crop = np.load(BytesIO(data['eigen_test_crop.npy']))
    print("Loaded.")
    return rgb, depth, crop
def DepthNorm(x, maxDepth):
    return maxDepth / x
def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25  ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()
    abs_rel = np.mean(np.abs(gt - pred) / gt)
    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())
    log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()
    return a1, a2, a3, abs_rel, rmse, log_10

In [ ]:
def predict(model, images, minDepth=10, maxDepth=1000):
    if len(images.shape) < 4:
        images = np.expand_dims(np.array(images), axis=-1)
        images = images.transpose((3, 2, 0, 1))
    else:
        images = images.transpose((0, 3, 1, 2))
    images = torch.from_numpy(images).float()/255
    with torch.no_grad():
        predictions = model(images)
    return np.clip(DepthNorm(predictions.cpu().numpy(), maxDepth=1000), minDepth, maxDepth) / maxDepth

In [ ]:
def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25  ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()
    abs_rel = np.mean(np.abs(gt - pred) / gt)
    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())
    log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()
    return a1, a2, a3, abs_rel, rmse, log_10

In [253]:
def my_predict(model, image, minDepth=10, maxDepth=1000):
  image = Image.fromarray(image, 'RGB')
  image = image.resize((640, 480))
  image = np.expand_dims(np.array(image), axis=-1)
  image = image.transpose((3, 2, 0, 1))
  image = torch.from_numpy(image).float()/255
  #image = image.cuda()
  with torch.no_grad():
    predictions = model(image)
  result = np.clip(DepthNorm(predictions.cpu().numpy(), maxDepth=maxDepth), minDepth, maxDepth) / maxDepth
  result = result.reshape(480,640)
  return result

In [ ]:
result = my_predict(model, rgb[0], minDepth=10, maxDepth=1000)
#Image.fromarray((result*255).astype(np.uint8))

In [254]:
depth_scores = np.zeros((len(rgb), 6)) # six metrics
for i in range(len(rgb)):
    result = my_predict(model, rgb[i])
    depth_scores[i] = compute_errors(depth[i], result*10)
errors = depth_scores.mean(axis=0)

In [261]:
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rmse', 'log_10'))
print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))

        a1,         a2,         a3,        rel,       rmse,     log_10
    0.7851,     0.9551,     0.9914,     0.1512,     0.5312,     0.0645


In [ ]:
rgb, depth, crop = load_test_data()

In [21]:
import segmentation_models_pytorch as smp
import torch
model = smp.Unet(encoder_name="timm-regnety_004",       
                encoder_weights="imagenet",     
                in_channels=3,                 
                classes=1           
                )

In [22]:
#dictt = torch.load('base_timm-regnety_004_model.pt')
dictt = torch.load('checkpoint_Regnet.pth')
model.load_state_dict(dictt['model_state_dict'] )
model.eval()
print("good")

good
